In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

tf.__version__

'2.9.2'

In [2]:
# Load in the data (takes about 5-6 minutes in Google Colab)
(train_data, test_data), ds_info = tfds.load(name="food101", # target dataset to get from TFDS
                                             split=["train", "validation"], # what splits of data should we get? note: not all datasets have train, valid, test
                                             shuffle_files=False, # shuffle files on download?
                                             as_supervised=True, # download data in tuple format (sample, label), e.g. (image, label)
                                             with_info=True) # include dataset metadata? if so, tfds.load() returns tuple (data, ds_info)

In [3]:
# Get class names
class_names = ds_info.features["label"].names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [5]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype from 'uint8' -> 'float32' and reshapes image to
  [img_shape, img_shape, color_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape]) # reshape to img_shape
  return tf.cast(image, tf.float32), label # return (float32_image, label) tuple

In [6]:
# Map preprocessing function to training data (and paralellize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [7]:
# Create TensorBoard callback (already have "create_tensorboard_callback()" from a previous notebook)
from helper_functions import create_tensorboard_callback

# Create ModelCheckpoint callback to save model's progress
checkpoint_path = "model_checkpoints/cp.ckpt" # saving weights requires ".ckpt" extension
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_accuracy", # save the model weights with best validation accuracy
                                                      save_best_only=True, # only save the best weights
                                                      save_weights_only=True, # only save model weights (not whole model)
                                                      verbose=0) # don't print out whether or not model is being saved

In [7]:
from tensorflow.keras import layers, mixed_precision
from tensorflow.keras.layers.experimental import preprocessing

# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB4(include_top=False)
base_model.trainable = False # freeze base model layers

# Create Functional model 
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
# x = preprocessing.Rescaling(1./255)(x)
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x) # want one output neuron per class 
# Separate activation of output layer so we can output float32 activations
outputs = layers.Activation(tf.keras.activations.softmax, dtype=tf.float32, name="softmax_float32")(x) 
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [8]:
# Check the layers in the base model and see what dtype policy they're using
for layer in model.layers[1].layers[:20]: # only check the first 20 layers to save output space
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 False float32 <Policy "float32">
rescaling False float32 <Policy "float32">
normalization False float32 <Policy "float32">
tf.math.truediv False float32 <Policy "float32">
stem_conv_pad False float32 <Policy "float32">
stem_conv False float32 <Policy "float32">
stem_bn False float32 <Policy "float32">
stem_activation False float32 <Policy "float32">
block1a_dwconv False float32 <Policy "float32">
block1a_bn False float32 <Policy "float32">
block1a_activation False float32 <Policy "float32">
block1a_se_squeeze False float32 <Policy "float32">
block1a_se_reshape False float32 <Policy "float32">
block1a_se_reduce False float32 <Policy "float32">
block1a_se_expand False float32 <Policy "float32">
block1a_se_excite False float32 <Policy "float32">
block1a_project_conv False float32 <Policy "float32">
block1a_project_bn False float32 <Policy "float32">
block1b_dwconv False float32 <Policy "float32">
block1b_bn False float32 <Policy "float32">


In [9]:
# Fit the feature extraction model for 5 epochs with tensorboard and model checkpoint callbacks
INITIAL_EPOCHS = 3

history_101_food_classes_feature_extract  = model.fit(train_data,
                                                      epochs=INITIAL_EPOCHS,
                                                      steps_per_epoch=len(train_data),
                                                      validation_data=test_data,
                                                      validation_steps=int(0.15 * len(test_data)),
                                                      callbacks=[model_checkpoint,
                                                                 create_tensorboard_callback(dir_name="training_logs",
                                                                                             experiment_name="efficientnetb4_101_classes_all_data_feature_extract")])

Saving TensorBoard log files to: training_logs/efficientnetb4_101_classes_all_data_feature_extract/20221024-121201
Epoch 1/3
2368/2368 [==============================] - 347s 142ms/step - loss: 1.7894 - accuracy: 0.5568 - val_loss: 1.2676 - val_accuracy: 0.6621
Epoch 2/3
2368/2368 [==============================] - 324s 137ms/step - loss: 1.3113 - accuracy: 0.6568 - val_loss: 1.1712 - val_accuracy: 0.6896
Epoch 3/3
2368/2368 [==============================] - 324s 137ms/step - loss: 1.1640 - accuracy: 0.6928 - val_loss: 1.1277 - val_accuracy: 0.6976


In [12]:
# Evaluate model (unsaved version) on whole test dataset
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

790/790 [==============================] - 66s 83ms/step - loss: 1.1272 - accuracy: 0.6950


[1.1271567344665527, 0.695049524307251]

In [13]:
# Save model locally (if you're using Google Colab, your saved model will Colab instance terminates)
model.save("food_vision_big_b4")  # SavedModel format

INFO:tensorflow:Assets written to: food_vision_big_b4\assets


INFO:tensorflow:Assets written to: food_vision_big_b4\assets


In [8]:
# Load model previously saved above
loaded_model = tf.keras.models.load_model("food_vision_big_b4")

In [15]:
# Check loaded model performance (this should be the same as results_feature_extract_model)
results_loaded_model = loaded_model.evaluate(test_data)
results_feature_extract_model, results_loaded_model

790/790 [==============================] - 84s 104ms/step - loss: 1.1272 - accuracy: 0.6951


([1.1271567344665527, 0.695049524307251],
 [1.127156376838684, 0.6950891017913818])

In [9]:
# Set all of the layers .trainable variable in the loaded model to True (so they're unfrozen)
for layer in loaded_model.layers:
    layer.trainable = True # set all layers to trainable

In [10]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
# Monitor the val_loss and stop training if it doesn't improve for 3 epochs
# See: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping for more
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)


# Create ModelCheckpoint callback to save best model during fine-tuning
# Save the best model only
# Monitor val_loss while training and save the best model (lowest val_loss)
# See: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint for more
checkpoint_path = "fine_tune_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

# Create ReduceLROnPlateau callback to reduce learning rate when a metric has stopped improving
# Once the validation loss stops improving for two or more epochs,
# we'll reduce the learning rate by a factor of 5 (e.g. 0.001 to 0.0002).
# And to make sure the learning rate doesn't get too low (and potentially result in our model learning nothing),
# we'll set the minimum learning rate to 1e-7.
# See: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 patience=2,
                                                 factor=0.2,  # multiply the learning rate by 0.2 (reduce by 5x)
                                                 verbose=1, # print out when learning rate goes down
                                                 min_lr=1e-7)

In [11]:
# Compile the model ready for fine-tuning
# Use the Adam optimizer with a 10x lower than default learning rate
loaded_model.compile(loss="sparse_categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                     optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate than the default
                     metrics=["accuracy"])

In [12]:
# Start to fine-tune (all layers)
# Use 100 epochs as the default
# Validate on 15% of the test_data
# Use the create_tensorboard_callback, ModelCheckpoint and EarlyStopping callbacks you created eaelier
# Start to fine-tune (all layers)
history_101_food_classes_b4_fine_tune = loaded_model.fit(train_data,
                                                         epochs=100, # fine-tune for a maximum of 100 epochs
                                                         steps_per_epoch=len(train_data),
                                                         validation_data=test_data,
                                                         validation_steps=int(0.15 * len(test_data)), # validation during training on 15% of test data
                                                         callbacks=[create_tensorboard_callback("training_logs", "efficientb4_101_classes_all_data_fine_tuning"), # track the model training logs
                                                                    model_checkpoint, # save only the best model during training
                                                                    early_stopping, # stop model after X epochs of no improvements
                                                                    reduce_lr]) # reduce the learning rate after X epochs of no improvements

Saving TensorBoard log files to: training_logs/efficientb4_101_classes_all_data_fine_tuning/20221024-125623
Epoch 1/100


NotFoundError: Graph execution error:

Detected at node 'model/efficientnetb4/block5b_expand_conv/Conv2D' defined at (most recent call last):
    File "C:\Tools\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Tools\Python\Python38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Gitrepo\TensorFlow_Developer\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Gitrepo\TensorFlow_Developer\venv\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "c:\Gitrepo\TensorFlow_Developer\venv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Gitrepo\TensorFlow_Developer\venv\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Tools\Python\Python38\lib\asyncio\b